# Shelly-Ann Duncan
# 11/17/22
# Using the Yelp API (Practice)

# Import necessary libraries

In [1]:
# imports
import os, json, math, time
import pandas as pd

from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

# Load credentials and create Yelp API Object

In [2]:
# load API credentials
with open('/Users/shell/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['Client-ID', 'API Key'])

In [3]:
# instantiate Yelp API variable
yelp = YelpAPI(login['API Key'], timeout_s = 5.0)

# Define search terms and filepaths

In [4]:
# set API call paramertes and filename before the first call
location = 'Brooklyn, NY 11236'
term = 'hamburgers'

In [5]:
# specify folder for saving data
FOLDER = 'Data/'
os.makedirs(FOLDER, exist_ok = True)

In [6]:
# specify JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER + f"{location.split(',')[0]}-{term}.json"
JSON_FILE

'Data/Brooklyn-hamburgers.json'

# Check if Json file exists and create if not

In [7]:
# check if the JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)

# if it doens't exist:
if file_exists == False:
    
    # create any needed folders
    # get the folder name only
    folder = os.path.dirname(JSON_FILE)
    
    # if JSON_FILE included a folder:
    if len(folder) > 0:
        # create the folder
        os.makedirs(folder, exist_ok = True)
        
        # inform user and save empty list
        print(f"(i) {JSON_FILE} not found. Saving empty list to file.")
        
        # save the first page of results
        with open(JSON_FILE, 'w') as f:
            json.dump([], f)

# if it exists, inform user
else:
    print(f"(i) {JSON_FILE} already exists.")    

(i) Data/Brooklyn-hamburgers.json already exists.


# Make the first API call to get the first page of data

In [21]:
# use the yelp_api variable's search_query method to perform our API call
results = yelp.search_query(location = 'Brooklyn', term = 'hamburgers')
type(results)

dict

In [22]:
# get the keys
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [23]:
# get the results for the business
results['businesses'][0]

{'id': 'Jy6qxXks0w5Wmurzu7rmBQ',
 'alias': '212-burgers-brooklyn-2',
 'name': '212 Burgers',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/wlBfz37uznsTRVOhvnHLlg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/212-burgers-brooklyn-2?adjust_creative=azSZ1YV457_fpKmHVQM9Bg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=azSZ1YV457_fpKmHVQM9Bg',
 'review_count': 133,
 'categories': [{'alias': 'burgers', 'title': 'Burgers'}],
 'rating': 4.5,
 'coordinates': {'latitude': 40.659118333958, 'longitude': -73.9816275578599},
 'transactions': ['delivery', 'pickup'],
 'price': '$$',
 'location': {'address1': '252 Prospect Park W',
  'address2': None,
  'address3': None,
  'city': 'Brooklyn',
  'zip_code': '11215',
  'country': 'US',
  'state': 'NY',
  'display_address': ['252 Prospect Park W', 'Brooklyn, NY 11215']},
 'phone': '+18883472747',
 'display_phone': '(888) 347-2747',
 'distance': 3474.971406888533}

In [24]:
# get the DataFrame
hamburgers0 = pd.DataFrame(results['businesses'])
hamburgers0.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,Jy6qxXks0w5Wmurzu7rmBQ,212-burgers-brooklyn-2,212 Burgers,https://s3-media1.fl.yelpcdn.com/bphoto/wlBfz3...,False,https://www.yelp.com/biz/212-burgers-brooklyn-...,133,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"{'latitude': 40.659118333958, 'longitude': -73...","[delivery, pickup]",$$,"{'address1': '252 Prospect Park W', 'address2'...",+18883472747,(888) 347-2747,3474.971407
1,bY0lygMoQaDoaCANGGeg_A,98-k-hamburger-brooklyn,98 K Hamburger,https://s3-media3.fl.yelpcdn.com/bphoto/cgGaPg...,False,https://www.yelp.com/biz/98-k-hamburger-brookl...,14,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",3.0,"{'latitude': 40.60394019123772, 'longitude': -...","[delivery, pickup]",NaN,"{'address1': '12 Bay 25th St', 'address2': '',...",,,6400.743980


In [25]:
# how many results total?
results['total']

6800

In [26]:
# how many businesses in our results
results_per_page = len(results['businesses'])
results_per_page

20

In [27]:
# use the math.cell to round up for the total number of pages of results
n_pages = math.ceil((results['total']) / results_per_page)
n_pages

340

In [28]:
for i in tqdm_notebook( range(1, n_pages + 1)):
   # the block of code we want to TRY to run
    try:
        
        # time.sleep(.2)
        
        # Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
            
        # save number of results for to use as offset
        n_results = len(previous_results)
        
        
        # use n_results as the OFFSET 
        results = yelp.search_query(location = location,
                                        term = term, 
                                        offset = n_results + 1)

        # append new results and save to file
        previous_results.extend(['businesses'])

        with open(JSON_FILE,'w') as f:
            json.dump(previous_results,f)
            
    # what to do if we get an error/exception.
    except Exception as e: # saving the error message so we can print it.
        print('[!] ERROR: ',e)

  0%|          | 0/340 [00:00<?, ?it/s]

In [30]:
# add the offset to the original api call
hamburgers_20 = yelp.search_query(location = 'Brooklyn',
                                  term = 'hamburgers',
                                  offset = 20)

In [36]:
# check the results
hamburgers_20 = pd.DataFrame(results['businesses'])
hamburgers_20.head(2)

""


In [34]:
# contatenate all pages
pages = pd.concat([hamburgers0, hamburgers_20],
                      ignore_index=True)
display(pages.head(3), pages.tail(3))

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,Jy6qxXks0w5Wmurzu7rmBQ,212-burgers-brooklyn-2,212 Burgers,https://s3-media1.fl.yelpcdn.com/bphoto/wlBfz3...,False,https://www.yelp.com/biz/212-burgers-brooklyn-...,133,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"{'latitude': 40.659118333958, 'longitude': -73...","[delivery, pickup]",$$,"{'address1': '252 Prospect Park W', 'address2'...",+18883472747,(888) 347-2747,3474.971407
1,bY0lygMoQaDoaCANGGeg_A,98-k-hamburger-brooklyn,98 K Hamburger,https://s3-media3.fl.yelpcdn.com/bphoto/cgGaPg...,False,https://www.yelp.com/biz/98-k-hamburger-brookl...,14,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",3.0,"{'latitude': 40.60394019123772, 'longitude': -...","[delivery, pickup]",NaN,"{'address1': '12 Bay 25th St', 'address2': '',...",,,6400.743980
2,HIGneWKny_s38Y6jRoTDrQ,moon-and-flower-house-brooklyn,Moon & Flower House,https://s3-media2.fl.yelpcdn.com/bphoto/SZWbYT...,False,https://www.yelp.com/biz/moon-and-flower-house...,75,"[{'alias': 'desserts', 'title': 'Desserts'}, {...",2.5,"{'latitude': 40.637238, 'longitude': -74.011715}","[delivery, pickup]",$$,"{'address1': '5912 7th Ave', 'address2': '', '...",+17187650020,(718) 765-0020,5707.596918


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
17,Y1sLUSnksXqjNgA3Mcx-yQ,dumont-burger-brooklyn,DuMont Burger,https://s3-media2.fl.yelpcdn.com/bphoto/onoAiA...,False,https://www.yelp.com/biz/dumont-burger-brookly...,771,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",3.5,"{'latitude': 40.71366, 'longitude': -73.96198}","[delivery, pickup]",$$,"{'address1': '314 Bedford Ave', 'address2': ''...",+17183846127,(718) 384-6127,7770.516609
18,Hc4_XRzCZ7kh-RpkB6c_qw,mr-chips-brooklyn,Mr Chips,https://s3-media1.fl.yelpcdn.com/bphoto/PrD3eI...,False,https://www.yelp.com/biz/mr-chips-brooklyn?adj...,10,"[{'alias': 'diners', 'title': 'Diners'}, {'ali...",4.5,"{'latitude': 40.63131688234249, 'longitude': -...","[delivery, pickup]",NaN,"{'address1': '7506 3rd Ave', 'address2': '', '...",+13479097054,(347) 909-7054,7192.381589
19,PY-v9ObQWyS1Oiz89IUfEw,skinflints-brooklyn,Skinflints,https://s3-media4.fl.yelpcdn.com/bphoto/AbbbEU...,False,https://www.yelp.com/biz/skinflints-brooklyn?a...,378,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"{'latitude': 40.626544, 'longitude': -74.024139}","[delivery, pickup]",$$,"{'address1': '7902 5th Ave', 'address2': '', '...",+17187451116,(718) 745-1116,6996.399368


In [37]:
# save the businesses as a records-oriented JSON file
(pages.to_json(orient='records'))

'[{"id":"Jy6qxXks0w5Wmurzu7rmBQ","alias":"212-burgers-brooklyn-2","name":"212 Burgers","image_url":"https:\\/\\/s3-media1.fl.yelpcdn.com\\/bphoto\\/wlBfz37uznsTRVOhvnHLlg\\/o.jpg","is_closed":false,"url":"https:\\/\\/www.yelp.com\\/biz\\/212-burgers-brooklyn-2?adjust_creative=azSZ1YV457_fpKmHVQM9Bg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=azSZ1YV457_fpKmHVQM9Bg","review_count":133,"categories":[{"alias":"burgers","title":"Burgers"}],"rating":4.5,"coordinates":{"latitude":40.659118334,"longitude":-73.9816275579},"transactions":["delivery","pickup"],"price":"$$","location":{"address1":"252 Prospect Park W","address2":null,"address3":null,"city":"Brooklyn","zip_code":"11215","country":"US","state":"NY","display_address":["252 Prospect Park W","Brooklyn, NY 11215"]},"phone":"+18883472747","display_phone":"(888) 347-2747","distance":3474.9714068885},{"id":"bY0lygMoQaDoaCANGGeg_A","alias":"98-k-hamburger-brooklyn","name":"98 K Hamburger","image_url":"https:\\/\\/